In [1]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import math
import pandas as pd
(x, N1) = librosa.load("DTMF_Lab2.wav", sr=None)

In [2]:
pi = math.pi

In [3]:
N = 8192

In [4]:
gap = int(N1 / 5)

In [5]:
#def hertzel(x, N):
#    g = [0 for i in range(N)]
#    g[N-1] = 0
#    g[N-2] = x[N-1]
#    k = 1633
#    for i in range(3, N):
#        g[N-i] = x[N-1] + 2 * math.cos(2 * pi * k / N) * g[N - i + 1] - g[N - i + 2]
#    X_sqared = g[0]**2 * math.cos(2 * pi * k / N) - g[1] - g[0] * math.sin(2 * pi * k / N)
#    print(X)
#hertzel(x[:8192], 8192)

In [6]:
x.size / N

25.83984375

In [7]:
def hertzel(x, N, k):
    s = [0 for i in range(N)]
    s[0] = x[0]
    s[1] = 2 * math.cos(2 * pi * k / N) + x[1]
    for i in range(2, N):
        s[i] = 2 * math.cos(2 * pi / k) * s[i-1] - s[i-2] + x[i]
    X_squared = s[N-1] ** 2 - 2 * math.cos(2 * pi / k) * s[N-1] * s[N-2] + s[N-2]**2
    return X_squared ** (1/2)
    
def find_ks(x, N):
    k_first_group = [697, 770, 852, 941]
    k_second_group = [1209, 1336, 1477, 1633]
    k_first = 0
    k_second = 0
    X_max = 0
    for k in k_first_group:
        X_k = hertzel(x, N, k)
        if (X_k > X_max):
            X_max = X_k
            k_first = k
    X_max = 0
    for k in k_second_group:
        X_k = hertzel(x, N, k)
        if (X_k > X_max):
            X_max = X_k
            k_second = k
    return (k_first, k_second)

In [8]:
table = pd.DataFrame(data = [[1, 2, 3, "A"], [4, 5, 6, "B"], [7, 8, 9, "C"], ["*", "0", "#", "D"]], 
             index=[697, 770, 852, 941], 
             columns=[1209, 1336, 1477, 1633])
table

,1209,1336,1477,1633
697,1,2,3,A
770,4,5,6,B
852,7,8,9,C
941,*,0,#,D


In [9]:
curPoint = 0
answer = []
while curPoint + gap < x.size:
    ks = find_ks(x[curPoint:curPoint + gap], 8192)
    answer.append(table.loc[ks[0], ks[1]])
    curPoint = curPoint + gap + gap // 2

In [10]:
answer

[1, 2, 6, 6, 2, 1, 2, 1, 2, 2, 6, 1, 9, 2, 6, 6]